In [2]:
import tweepy
import pandas as pd


In [3]:
api_key = 'ALR6ExjYkMBfebl33tcjXyZ4s'
api_key_secret = 'nDDTt61clONQ8isJifsLT7nqhxfjHFFlo6W8rl2SblhPMeVLk3'

access_token = '2317868736-6BMg18lkng6EIY2nrSSoISburY9QcaUovk78xPY'
access_token_secret = 'UcHtfXjvurM4dT9ZpMgvwDfZ2ukQtEGQqaLEfhueSeFyq'

auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)



limit=500

keywords = '#HaltDieFresse'

tweets = tweepy.Cursor(api.search_tweets, q=keywords, count=100, tweet_mode='extended').items(limit)

columns = ['Tweet']
data = []

for tweet in tweets:
    data.append([tweet.full_text])

df = pd.DataFrame(data, columns=columns)
df.to_csv('#HaltDieFresse_tweets.csv')



keywords = '#heute'

tweets = tweepy.Cursor(api.search_tweets, q=keywords, count=100, tweet_mode='extended').items(limit)

columns = ['Tweet']
data = []

for tweet in tweets:
    data.append([tweet.full_text])

df = pd.DataFrame(data, columns=columns)
df.to_csv('#heute_tweets.csv')



keywords = '#DankeMerkel'

tweets = tweepy.Cursor(api.search_tweets, q=keywords, count=100, tweet_mode='extended').items(limit)

columns = ['Tweet']
data = []

for tweet in tweets:
    data.append([tweet.full_text])

df = pd.DataFrame(data, columns=columns)
df.to_csv('#DankeMerkel_tweets.csv')



keywords = 'Fresse'

tweets = tweepy.Cursor(api.search_tweets, q=keywords, count=100, tweet_mode='extended').items(limit)


columns = ['Tweet']
data = []

for tweet in tweets:
    data.append([tweet.full_text])

df = pd.DataFrame(data, columns=columns)
df.to_csv('Fresse_tweets.csv')



keywords = 'heute'

tweets = tweepy.Cursor(api.search_tweets, q=keywords, count=100, tweet_mode='extended').items(limit)

columns = ['Tweet']
data = []

for tweet in tweets:
    data.append([tweet.full_text])

df = pd.DataFrame(data, columns=columns)
df.to_csv('heute_tweets.csv')



keywords = 'danke Merkel'

tweets = tweepy.Cursor(api.search_tweets, q=keywords, count=100, tweet_mode='extended').items(limit)

columns = ['Tweet']
data = []

for tweet in tweets:
    data.append([tweet.full_text])

df = pd.DataFrame(data, columns=columns)
df.to_csv('danke-Merkel_tweets.csv')




                                                 Tweet
0    RT @v_schleicher: Die Linken sind eindeutig au...
1    RT @seokjinnieDE: Vor genau 2 Jahren wurde das...
2    War nicht gut heute. Passqoute unterirdisch. D...
3    @ISDT Heute war es von mir aus einer, im Hinsp...
4    RT @nenacasc: Wenn die Maskenpflicht einmal fä...
..                                                 ...
295  RT @FLietzke: Heute gibt es eine Pilspfanne. h...
296  RT @martel_afd: Alles Gute unserem Ehrenvorsit...
297  RT @semanthis: Gemäßigt und realistisch sind d...
298  RT @Team_Stollberg: Auch heute wird durch die ...
299  RT @BobGerbilkiller: @Karl_Lauterbach Hallo He...

[300 rows x 1 columns]
